In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA


In [10]:
df_origin = pd.read_csv("datas/5min_MYM_量價+YM+ES+VIX+ZN+RTY+NQ+DX資料_class.csv")

In [22]:
# 假设您的数据存储在一个名为'df'的pandas DataFrame中
numeric_cols = df_origin.select_dtypes(include=["number"]).columns
filtered_rows = df_origin[(df_origin[numeric_cols] > 999999999).any(axis=1)]
data_df_清洗 = filtered_rows.dropna().drop(["Unnamed: 0"], axis=1)
data_df_清洗.set_index("datetime", inplace=True)

In [23]:
data_df_target = data_df_清洗[["future_5min_change","future_10min_change","future_15min_change","future_30min_change","future_60min_change","future_90min_change","future_120min_change","future_180min_change","future_240min_change"]]
data_df_feature = data_df_清洗.drop(["future_5min_change","future_10min_change","future_15min_change","future_30min_change","future_60min_change","future_90min_change","future_120min_change","future_180min_change","future_240min_change"], axis=1)

In [27]:
df = data_df_feature.copy()

# 标准化数据
X = (df - df.mean()) / df.std()

# 进行PCA降维
pca = PCA()
X_pca = pca.fit_transform(X)

# 选择合适的主成分数量
explained_variance_ratio = pca.explained_variance_ratio_
n_components = next(i for i, v in enumerate(np.cumsum(explained_variance_ratio)) if v >= 0.95)

# 将数据转换到新的PCA空间
X_reduced = X_pca[:, :n_components]

# 将降维后的数据保存到新的DataFrame
df_reduced = pd.DataFrame(X_reduced, columns=[f'PC{i+1}' for i in range(n_components)], index=df.index)

# 您现在可以使用df_reduced进行后续的数据分析和建模
print(f'原始数据有{df.shape[1]}个特征,降维后只有{n_components}个主成分,保留了95%以上的方差信息。')

原始数据有130个特征,降维后只有37个主成分,保留了95%以上的方差信息。


In [32]:
data_PCAfinish = data_df_target.merge(df_reduced, on="datetime", how="left")

In [33]:
data_PCAfinish.to_csv("datas/5min_MYM_量價+YM+ES+VIX+ZN+RTY+NQ+DX資料_class_PCA.csv")